# isolated_atom - Python class

__Python imports__

- [numpy](http://www.numpy.org/)
- [IPython](https://ipython.org)
- [atomman](https://github.com/usnistgov/atomman)
- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from pathlib import Path
import datetime

# http://www.numpy.org/
import numpy as np

# https://ipython.org/
from IPython.display import display, Markdown

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

Notebook last executed on 2022-03-07 using iprPy version 0.11.2


## 1. Load calculation and view description

### 1.1. Load the calculation

In [2]:
# Load the calculation being demoed
calculation = iprPy.load_calculation('isolated_atom')

### 1.2. Display calculation description and theory

In [3]:
# Display main docs and theory
display(Markdown(calculation.maindoc))
display(Markdown(calculation.theorydoc))

# isolated_atom calculation style

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

## Introduction

The isolated_atom calculation style evaluates the base energies of all atomic models associated with an interatomic potential. For some potentials, the isolated energy values are necessary to properly compute the cohesive energy of crystal structures.  This also provides a simple test whether a potential implementation is compatible with a version of LAMMPS.

### Version notes

- 2020-09-22: Notebook first added.
- 2022-02-16: Notebook updated to reflect version 0.11.

### Additional dependencies

### Disclaimers

- [NIST disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)
- Some potentials have two cutoffs with atomic energies outside the first being the "isolated" energy while outside the second have zero energy.  The first isolated energy values for those potentials can be found using the diatom_scan calculation instead.


## Method and Theory

The calculation loops over all symbol models of the potential and creates a system with a single particle inside a system with non-periodic boundary conditions.  The potential energy of each unique isolated atom is evaluated without relaxation/integration.

The cohesive energy, $E_{coh}$, of a crystal structure is given as the per-atom potential energy of the crystal structure at equilibrium $E_{crystal}/N$ relative to the potential energy of the same atoms infinitely far apart, $E_i^{\infty}$

$$ E_{coh} = \frac{E_{crystal} - \sum{N_i E_{i}^{\infty}}}{N},$$

Where the $N_i$ values are the number of each species $i$ and $\sum{N_i} = N$.

For most potentials, $E_i^{\infty}=0$ meaning that the measured potential energy directly corresponds to the cohesive energy.  However, this is not the case for all potentials as some have offsets either due to model artifacts or because it allowed for a better fitted model.


## 2. Specify input parameters

The calculation input parameters can be specified directly to the associated calculation's attributes.  Note that these parameters are often part of calculation subsets allowing multiple calculations to share the same input terms.

### 2.1. LAMMPS-related commands

Calculations that use LAMMPS need to know which LAMMPS executable to use.  Additionally, running LAMMPS in parallel requires specifying an MPI command.

- __commands__ is a LammpsCommands subset.
    - __lammps_command__ is the LAMMPS command to use.
    - __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [4]:
calculation.commands.lammps_command = 'lmp_serial'
calculation.commands.mpi_command = None

### 2.2. Interatomic potential

LAMMPS calculations also need to know which potential to use.  These are handled by loaded potentials/atomman PotentialLAMMPS objects.

- __potential__ is a LammpsPotential subset.
    - __potential__ is a loaded PotentialLAMMPS or PotentialLAMMPSKIM object.

In [5]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
potential = am.load_lammps_potential(id=potential_name, getfiles=True)

# So many "potential"s...
calculation.potential.potential = potential

## 3. Preview the data record

Optionally, an incomplete JSON/XML record can be generated for the calculation after setting the inputs.

- __build_model()__ will interpret the object's attributes into a data model.
- __model__ is the generated data model.
- __model.json()__ will convert the model to JSON.
- __model.xml()__ will convert the model to XML.

In [6]:
calculation.build_model()
print(calculation.model.json(indent=4))

{
    "calculation-isolated-atom": {
        "key": "2f434d6a-ee16-42d9-a451-6f658429de8b",
        "calculation": {
            "iprPy-version": "0.11.2",
            "atomman-version": "1.4.3",
            "LAMMPS-version": "3 Mar 2020",
            "script": "calc_isolated_atom",
            "branch": "main"
        },
        "potential-LAMMPS": {
            "key": "a7c9b786-5aa7-481b-86eb-7e4edd9cec02",
            "id": "1999--Mishin-Y--Ni--LAMMPS--ipr1",
            "potential": {
                "key": "81adb388-59eb-4f49-88bf-6f06a1343fae",
                "id": "1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Ni"
            }
        },
        "status": "not calculated"
    }
}


## 4. Run calculation and view results

### 4.1. Run calculation
 
__run()__ Runs the calculation using the current object attribute values or supplied parameters. Status after running will be either "finished" or "error".
        
- __params__ (*dict, str or file-like object, optional*) The key-value parameter file or dict of associated values to read in.  If not given, will run based on the current object attribute values.
- __newkey__ (*bool, optional*) If True, then the calculation's key and name will be replaced with a new UUID4.  This allows for iterations on previous runs to be uniquely labeled.  Default value is False.
- __results_json__ (*bool, optional*) If True, then a "results.json" file will be generated following the run.
- __verbose__ (*bool, optional*) If True, a message relating to the calculation's status will be printed upon completion.  Default value is False.

In [7]:
calculation.run(verbose=True)

Calculation finished successfully


### 4.2. Results attributes

- __isolated_atom_energy__ (*dict*) contains the computed isolated atom energies for each of the potential's symbol models.  

In [8]:
energy_unit = 'eV'
for symbol, energy in calculation.isolated_atom_energy.items():
    print(symbol, uc.get_in_units(energy, energy_unit), energy_unit)

Ni -3.0970029925997e-11 eV


### 4.3. Output as JSON/XML

In [9]:
calculation.build_model()
print(calculation.model.json(indent=4))

{
    "calculation-isolated-atom": {
        "key": "2f434d6a-ee16-42d9-a451-6f658429de8b",
        "calculation": {
            "iprPy-version": "0.11.2",
            "atomman-version": "1.4.3",
            "LAMMPS-version": "3 Mar 2020",
            "script": "calc_isolated_atom",
            "branch": "main"
        },
        "potential-LAMMPS": {
            "key": "a7c9b786-5aa7-481b-86eb-7e4edd9cec02",
            "id": "1999--Mishin-Y--Ni--LAMMPS--ipr1",
            "potential": {
                "key": "81adb388-59eb-4f49-88bf-6f06a1343fae",
                "id": "1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Ni"
            }
        },
        "isolated-atom-energy": {
            "symbol": "Ni",
            "energy": {
                "value": -3.0970029925997e-11,
                "unit": "eV"
            }
        }
    }
}
